# List of Exposures  at USDF in OGA


- work with Weakly_2023_44
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/13
- last update : 2024/02/01



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_44/analysis_tools/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/cp_pipe/g9dd2c6f9d4+29174bd147/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/shapelet/g44018dc512+2565ec44ba/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_modelfit/gd162630629+22efd3ad40/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet_lite/g13f254ea0d+81bc2a20b4/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/proxmin/g33b4157f25/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet/g63ff1f842d+4083830bf8/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_scarlet/gabf74fab76+76066ce010/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_p

In [3]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2023_44/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_44/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_44/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2023_44
   g4213664e8e+d053631def 	current w_2023_44 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    try:
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd, info_timespan_begin_mjd  ]
 
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [14]:
info

exposure.RecordClass(instrument='LATISS', id=2024013100297, physical_filter='empty~blue300lpmm_qn1', obs_id='AT_O_20240131_000297', exposure_time=60.0, dark_time=60.235, observation_type='science', observation_reason='x_offset_50', day_obs=20240131, seq_num=297, seq_start=297, seq_end=297, group_name='2024-02-01T02:09:59.159', group_id=2866829991590000, target_name='HD38666', science_program='spec-survey', tracking_ra=86.50931002143818, tracking_dec=-32.29492692072391, sky_angle=328.1492128050928, azimuth=245.87487685781213, zenith_angle=5.302478707252348, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2024-02-01 02:10:19.224013', scale='tai', format='iso'), end=astropy.time.Time('2024-02-01 02:11:19.459000', scale='tai', format='iso')))

In [15]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76553,2024013100293,AT_O_20240131_000293,20240131,293,2024-02-01 02:04:53.135016,2024-02-01 02:05:53.375,science,HD38666,empty~holo4_003,4.266205,60.0,86.495513,-32.302349,328.148951,240.156849,4.266205,spec-survey,2.460342e+06,60341.086726
76554,2024013100294,AT_O_20240131_000294,20240131,294,2024-02-01 02:06:06.238019,2024-02-01 02:07:06.470,science,HD38666,empty~holo4_003,4.496318,60.0,86.495638,-32.30228,328.148686,241.674792,4.496318,spec-survey,2.460342e+06,60341.087572
76555,2024013100295,AT_O_20240131_000295,20240131,295,2024-02-01 02:07:29.317995,2024-02-01 02:08:29.550,science,HD38666,empty~blue300lpmm_qn1,4.761941,60.0,86.495588,-32.302322,328.148725,243.21126,4.761941,spec-survey,2.460342e+06,60341.088534
76556,2024013100296,AT_O_20240131_000296,20240131,296,2024-02-01 02:08:54.405001,2024-02-01 02:09:54.641,science,HD38666,empty~blue300lpmm_qn1,5.051058,60.0,86.481791,-32.309724,328.148761,244.579795,5.051058,spec-survey,2.460342e+06,60341.089519


In [16]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76553,2024013100293,AT_O_20240131_000293,20240131,293,2024-02-01 02:04:53.135016,2024-02-01 02:05:53.375,science,HD38666,empty~holo4_003,4.266205,60.0,86.495513,-32.302349,328.148951,240.156849,4.266205,spec-survey,2.460342e+06,60341.086726
76554,2024013100294,AT_O_20240131_000294,20240131,294,2024-02-01 02:06:06.238019,2024-02-01 02:07:06.470,science,HD38666,empty~holo4_003,4.496318,60.0,86.495638,-32.30228,328.148686,241.674792,4.496318,spec-survey,2.460342e+06,60341.087572
76555,2024013100295,AT_O_20240131_000295,20240131,295,2024-02-01 02:07:29.317995,2024-02-01 02:08:29.550,science,HD38666,empty~blue300lpmm_qn1,4.761941,60.0,86.495588,-32.302322,328.148725,243.21126,4.761941,spec-survey,2.460342e+06,60341.088534
76556,2024013100296,AT_O_20240131_000296,20240131,296,2024-02-01 02:08:54.405001,2024-02-01 02:09:54.641,science,HD38666,empty~blue300lpmm_qn1,5.051058,60.0,86.481791,-32.309724,328.148761,244.579795,5.051058,spec-survey,2.460342e+06,60341.089519


## Selection of science

In [18]:
df_science = df_exposure[df_exposure.type == 'science']

In [19]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_8429/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [20]:
df_science.head(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
657,2022091200104,AT_O_20220912_000104,20220912,104,2022-09-13 01:20:16.390009,2022-09-13 01:20:46.618,science,HD167060,empty~ronchi170lpmm,35.761102,30.0,274.485264,-61.742148,130.613923,200.833163,35.761102,unknown,2.459836e+06,59835.055745
650,2022091200105,AT_O_20220912_000105,20220912,105,2022-09-13 01:20:59.277003,2022-09-13 01:21:29.513,science,HD167060,empty~ronchi170lpmm,35.816568,30.0,274.485275,-61.742168,130.613889,200.943213,35.816568,unknown,2.459836e+06,59835.056242
652,2022091200106,AT_O_20220912_000106,20220912,106,2022-09-13 01:21:48.291985,2022-09-13 01:22:18.520,science,HD167060,empty~holo4_003,35.884569,30.0,274.485183,-61.74221,130.613866,201.067976,35.884569,unknown,2.459836e+06,59835.056809
654,2022091200107,AT_O_20220912_000107,20220912,107,2022-09-13 01:22:31.094006,2022-09-13 01:23:01.329,science,HD167060,empty~holo4_003,35.940561,30.0,274.485217,-61.74215,130.613861,201.176876,35.940561,unknown,2.459836e+06,59835.057304
660,2022091200111,AT_O_20220912_000111,20220912,111,2022-09-13 01:24:55.866989,2022-09-13 01:25:26.113,science,HD2811,empty~ronchi170lpmm,52.803456,30.0,7.828469,-43.566523,351.230009,123.786942,52.803456,unknown,2.459836e+06,59835.058980
658,2022091200112,AT_O_20220912_000112,20220912,112,2022-09-13 01:25:38.685988,2022-09-13 01:26:08.920,science,HD2811,empty~ronchi170lpmm,52.677652,30.0,7.828567,-43.566534,351.229973,123.762716,52.677652,unknown,2.459836e+06,59835.059476
668,2022091200113,AT_O_20220912_000113,20220912,113,2022-09-13 01:26:27.713000,2022-09-13 01:26:57.947,science,HD2811,empty~holo4_003,52.535234,30.0,7.828656,-43.566495,351.229866,123.734754,52.535234,unknown,2.459836e+06,59835.060043
665,2022091200114,AT_O_20220912_000114,20220912,114,2022-09-13 01:27:10.524999,2022-09-13 01:27:40.765,science,HD2811,empty~holo4_003,52.40619,30.0,7.828528,-43.566516,351.229887,123.710487,52.40619,unknown,2.459836e+06,59835.060538
661,2022091200119,AT_O_20220912_000119,20220912,119,2022-09-13 01:29:43.197999,2022-09-13 01:30:13.443,science,HD185975,empty~ronchi170lpmm,57.111008,30.0,308.018105,-87.47647,85.868273,179.55936,57.111008,unknown,2.459836e+06,59835.062306
666,2022091200120,AT_O_20220912_000120,20220912,120,2022-09-13 01:30:26.020016,2022-09-13 01:30:56.245,science,HD185975,empty~ronchi170lpmm,57.109986,30.0,308.018207,-87.47646,85.868022,179.568978,57.109986,unknown,2.459836e+06,59835.062801


In [21]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
76196,2024013000139,AT_O_20240130_000139,20240130,139,2024-01-31 03:49:13.632018,2024-01-31 03:49:43.884,science,Photo0800-1_003,SDSSg_65mm~empty,8.072941,30.0,122.467324,-36.206365,359.999819,140.034217,8.072941,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.159186
76197,2024013000140,AT_O_20240130_000140,20240130,140,2024-01-31 03:50:00.474009,2024-01-31 03:50:30.700,science,Photo0800-1_003,SDSSr_65mm~empty,7.967227,30.0,122.467347,-36.206361,359.999708,140.845601,7.967227,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.159728
76198,2024013000141,AT_O_20240130_000141,20240130,141,2024-01-31 03:50:46.968989,2024-01-31 03:51:17.210,science,Photo0800-1_003,SDSSz_65mm~empty,7.858892,30.0,122.467338,-36.206387,359.999979,141.717085,7.858892,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.160266
76199,2024013000142,AT_O_20240130_000142,20240130,142,2024-01-31 03:51:35.419018,2024-01-31 03:52:05.661,science,Photo0800-1_003,empty~SDSSy_65mm,7.752459,30.0,122.467174,-36.20626,0.000075,142.614144,7.752459,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.160827
76200,2024013000143,AT_O_20240130_000143,20240130,143,2024-01-31 03:53:01.830016,2024-01-31 03:53:32.073,science,Photo0800-1_004,SDSSg_65mm~empty,7.574863,30.0,122.485584,-36.206292,359.99978,144.219285,7.574863,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.161827
76201,2024013000144,AT_O_20240130_000144,20240130,144,2024-01-31 03:53:44.813005,2024-01-31 03:54:15.048,science,Photo0800-1_004,SDSSr_65mm~empty,7.485208,30.0,122.48568,-36.206383,359.999794,145.086266,7.485208,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.162324
76202,2024013000145,AT_O_20240130_000145,20240130,145,2024-01-31 03:54:31.465981,2024-01-31 03:55:01.703,science,Photo0800-1_004,SDSSz_65mm~empty,7.391206,30.0,122.485578,-36.206289,359.999994,146.03817,7.391206,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.162864
76203,2024013000146,AT_O_20240130_000146,20240130,146,2024-01-31 03:55:19.853005,2024-01-31 03:55:50.086,science,Photo0800-1_004,empty~SDSSy_65mm,7.293399,30.0,122.485305,-36.206005,359.999533,147.08025,7.293399,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.163424
76204,2024013000147,AT_O_20240130_000147,20240130,147,2024-01-31 03:56:46.055997,2024-01-31 03:57:16.292,science,Photo0800-1_005,SDSSg_65mm~empty,7.137718,30.0,122.503965,-36.206375,0.000159,148.880934,7.137718,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.164422
76205,2024013000148,AT_O_20240130_000148,20240130,148,2024-01-31 03:57:32.897988,2024-01-31 03:58:03.128,science,Photo0800-1_005,SDSSr_65mm~empty,7.051381,30.0,122.503947,-36.206334,359.999742,149.955143,7.051381,AUXTEL_PHOTO_IMAGING,2.460341e+06,60340.164964


In [22]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718,
       20230720, 20230801, 20230802, 20230803, 20230815, 20230816,
       20230817, 20230829, 20230912, 20230913, 20230914, 20230926,
       20230927, 20230928, 20231010, 20231011, 20231107, 20231108,
       20231113, 20231114, 20231115, 20231116, 20231120, 20231121,
       20231127, 20231128, 20231129, 20231130, 20231204, 20231205,
       20231206, 20231207, 20231211, 20231212, 20231214, 20231215,
       20231219, 20231220, 20231221, 20231222, 20240129, 20240

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [23]:

# Januaryr 2024
#date_sel = 20240129
#date_sel = 20240130
#date_sel = 20240131
date_sel = 20230928

## Query Dimension records on Exposures

In [24]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [25]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [26]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [27]:
len(df_science_selected)

5

# List of filters

In [28]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'empty~blue300lpmm_qn1'], dtype=object)

In [29]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [30]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [31]:
pd.set_option('display.max_rows', None)

In [32]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20240131_filt_empty-holo4_003.list


/tmp/ipykernel_8429/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [33]:
N = len(all_def_science_selected_final)

In [34]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
76553,2024013100293,AT_O_20240131_000293,20240131,293,2024-02-01 02:04:53.135016,2024-02-01 02:05:53.375,science,HD38666,empty~holo4_003,4.266205,60.0,86.495513,-32.302349,328.148951,240.156849,4.266205,spec-survey,2.460342e+06,60341.086726
76554,2024013100294,AT_O_20240131_000294,20240131,294,2024-02-01 02:06:06.238019,2024-02-01 02:07:06.470,science,HD38666,empty~holo4_003,4.496318,60.0,86.495638,-32.30228,328.148686,241.674792,4.496318,spec-survey,2.460342e+06,60341.087572


In [35]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [36]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [37]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [38]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [39]:
df_forsave = pd.concat(all_def_science_selected_final)

In [40]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [41]:
filename_out_csv

'holosummary_20240131.csv'

In [42]:
df_forsave.to_csv(filename_out_csv) 

In [43]:
#! pip install openpyxl

In [44]:
df_forsave.to_excel(filename_out_excel) 